# Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings('ignore')

# Data

In [ ]:
boston_data = datasets.load_boston()
data = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
target = pd.DataFrame(boston_data['target'], columns=['Target'])
df = pd.concat([data, target], axis=1)

In [ ]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# Split data

In [ ]:
X = df.drop(['Target'],axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 100)

# LGBM + GridSearchCV

In [ ]:
#모델 생성
model=LGBMRegressor()

#최적화할 파라미터 범위 
lgbm_params = {'learning_rate': [0.05, 0.1, 0.2],
               'max_depth': [6, 8, 10],
               'n_estimator': [25, 50, 100]
              }
            
grid_lgbm = GridSearchCV(model, param_grid=lgbm_params, scoring = 'neg_root_mean_squared_error', cv=5, n_jobs=-1)
grid_lgbm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [6, 8, 10],
                         'n_estimator': [25, 50, 100]},
             scoring='neg_root_mean_squared_error')

In [ ]:
result_df = pd.DataFrame(grid_lgbm.cv_results_)

In [ ]:
result_df[['params', 'mean_test_score', 'rank_test_score']].head()

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",-3.625503,25
1,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",-3.625503,25
2,"{'learning_rate': 0.05, 'max_depth': 6, 'n_est...",-3.625503,25
3,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",-3.619443,19
4,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",-3.619443,19


In [ ]:
print('best params:', grid_lgbm.best_params_)
print('best score:', grid_lgbm.best_score_)

best params: {'learning_rate': 0.2, 'max_depth': 8, 'n_estimator': 25}
best score: -3.463534826116618


# Prediction

In [ ]:
pred = grid_lgbm.predict(X_test)

#RMSE 
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : {}".format(rmse))

RMSE : 3.9376135555099556
